# Great Circle

```{warning} This is not meant to be a standalone notebook.
This notebook is part of the process we have for adding entries to the NCL Index and is not meant to be used as tutorial or example code.
```

## Functions covered
- [area_poly_sphere](https://www.ncl.ucar.edu/Document/Functions/Built-in/area_poly_sphere.shtml)
- [css2c](https://www.ncl.ucar.edu/Document/Functions/Built-in/css2c.shtml)
- [csc2s](https://www.ncl.ucar.edu/Document/Functions/Built-in/csc2s.shtml)
- [gc_onarc](https://www.ncl.ucar.edu/Document/Functions/Built-in/gc_onarc.shtml)

## NCL code

```{literalinclude} ../ncl_raw/great_circle.ncl

```

## Python Functionality

### area_poly_sphere

In [1]:
import math

ncl_results = {}

poly_name = [
    "Boulder, Boston, Houston",
    "Four Corners of Colorado",
    "Caltech, Alberta, Greenwich, Paris, Madrid",
    "Crossing the Equator",
    "Crossing the Prime Meridian",
    "Half of the World",
    "Single Point -> Invalid NCL",
    "Single Degree",
]

ncl_lat = [
    [40.0150, 42.3601, 29.5518],
    [41.00488, 41.00203, 37.00540, 37.00051],
    [34.1377, 53.9333, 51.4934, 48.8575, 40.4167],
    [0.4078, -5.9230, 0.1864],
    [53.3498, 52.3676, 51.9939],
    [90.0, 0.0, -90.0, 0.0],
    [40, 40, 40, 40],
    [40, 40, 41, 41],
]
ncl_lon = [
    [-105.2705, -71.0589, -95.0982],
    [-109.05001, -102.05348, -103.04633, -109.04720],
    [-118.1253, -116.5765, 0.0098, 2.3514, -3.7033],
    [9.4403, 12.3636, 6.6131],
    [-6.2603, 4.9041, -4.9760],
    [0.0, -90.0, 0.0, 90.0],
    [-105, -105, -105, -105],
    [-105, -106, -106, -105],
]
ncl_results[poly_name[0]] = 1940856
ncl_results[poly_name[1]] = 250007.6
ncl_results[poly_name[2]] = 11634800
ncl_results[poly_name[3]] = 114894.8
ncl_results[poly_name[4]] = 54450.39
ncl_results[poly_name[5]] = 255032000
ncl_results[poly_name[6]] = -127516000
ncl_results[poly_name[7]] = 9401.705

from pyproj import Geod

python_results = {}
geod = Geod(ellps="sphere")  # radius = 6370997 m
for i in range(len(poly_name)):
    poly_area_m, _ = geod.polygon_area_perimeter(ncl_lon[i], ncl_lat[i])
    poly_area_km2 = abs(poly_area_m) * 1e-6
    python_results[poly_name[i]] = poly_area_km2

### css2c

In [2]:
import geocat.datafiles as gdf
import numpy as np
from astropy.coordinates.representation import UnitSphericalRepresentation
from astropy import units

css2c_data = gdf.get('applications_files/ncl_outputs/css2c_output.txt')
css2c_data = np.loadtxt(css2c_data, delimiter=',', skiprows=6)

lat_lon = tuple(map(tuple, (css2c_data[::, 0:2])))
cart_values = tuple(css2c_data[::, 2:])
ncl_css2c = dict(zip(lat_lon, cart_values))

Updating file 'registry.txt' from 'https://github.com/NCAR/GeoCAT-datafiles/raw/main/registry.txt' to '/home/cs/.cache/geocat'.


In [3]:
## Collect Latitude and Longtiude Pairs
lat_lon = []
for lat in range(-90, 90 + 1):
    for lon in range(-180, 180 + 1):
        lat_lon.append((lat, lon))

## Calculate Cartesian coordinates
astropy_css2c = {}
for i, pair in enumerate(lat_lon):
    lat, lon = pair
    spherical_coords = UnitSphericalRepresentation(
        lat=lat * units.deg, lon=lon * units.deg
    )
    cart_coords = spherical_coords.to_cartesian()
    astropy_css2c[pair] = cart_coords.xyz.value

### csc2s

In [4]:
import geocat.datafiles as gdf
from astropy.coordinates.representation import (
    CartesianRepresentation,
    SphericalRepresentation,
)
import numpy as np

csc2s_data = gdf.get('applications_files/ncl_outputs/csc2s_output.txt')
csc2s_data = np.loadtxt(csc2s_data, delimiter=',', skiprows=6)

input_lat_lon = tuple(map(tuple, csc2s_data[::, 0:2]))
cart_values = tuple(map(tuple, (csc2s_data[::, 2:5])))
output_lat_lon = tuple(map(tuple, (csc2s_data[::, 5:])))
ncl_csc2s = dict(zip(input_lat_lon, cart_values))
ncl_csc2s_input_output = dict(zip(input_lat_lon, output_lat_lon))

In [5]:
## Calculate Spherical coordinates
def spherical_cart(x, y, z):
    cart_coords = CartesianRepresentation(x=x, y=y, z=z)
    spherical_coords = cart_coords.represent_as(SphericalRepresentation)
    # convert latitude/longitude from radians to degrees
    lat_deg = np.rad2deg(spherical_coords.lat.value)
    lon_deg = (
        np.rad2deg(spherical_coords.lon.value) + 180
    ) % 360 - 180  # keep longitude between -180 to 180
    return (lat_deg, lon_deg)


astropy_csc2s = {}
for xyz in cart_values:
    x, y, z = xyz
    lat_lon = spherical_cart(x, y, z)
    astropy_csc2s[lat_lon] = tuple(xyz)

### gc_onarc

In [6]:
ncl_gc_onarc = {}

# Arc above/below latitude with point in Boulder with same longitude = True
ncl_lat = (40.0150, 50.0150, 30.0150)
ncl_lon = (-105.2705, -105.2705, -105.2705)
ncl_gc_onarc[(ncl_lat, ncl_lon)] = True

# Arc from Denver to Boston, point with Boulder
ncl_lat = (40.0150, 39.73915, 42.35843)
ncl_lon = (-105.2705, -104.9847, -71.05977)
ncl_gc_onarc[(ncl_lat, ncl_lon)] = False

# All three points in the same position
ncl_lat = (40.0150, 40.0150, 40.0150)
ncl_long = (-105.2705, -105.2705, -105.2705)
ncl_gc_onarc[(ncl_lat, ncl_long)] = True

# Anti-meridian
ncl_lat = (24.86658349036834, 55.182644873824785, 49.05673786129486)
ncl_lon = (151.60735730237568, 30.6703372467811, 179.59874545906715)
ncl_gc_onarc[(ncl_lat, ncl_lon)] = False

In [7]:
import numpy as np


def latlon_to_cart(lat, lon):
    from astropy.coordinates.representation import UnitSphericalRepresentation
    from astropy import units

    spherical_coords = UnitSphericalRepresentation(
        lat=lat * units.deg, lon=lon * units.deg
    )
    cart_coords = spherical_coords.to_cartesian()
    return np.array([cart_coords.x, cart_coords.y, cart_coords.z])


python_onarc = {}

from uxarray.grid.arcs import point_within_gca

for ncl_latlon in ncl_gc_onarc.keys():
    pt_within = latlon_to_cart(ncl_latlon[0][0], ncl_latlon[1][0])
    vertex_a = latlon_to_cart(ncl_latlon[0][1], ncl_latlon[1][1])
    vertex_b = latlon_to_cart(ncl_latlon[0][2], ncl_latlon[1][2])

    python_onarc[ncl_latlon] = point_within_gca(pt_within, vertex_a, vertex_b)

## Comparison

### area_poly_sphere

<div class="admonition alert alert-info">
    <p class="admonition-title" style="font-weight:bold">Important Note</p>
    NCL does not return a valid value for a single point ("Single Point -> Invalid NCL") where Python does, so this error is ignored below
</div>

In [8]:
for key in ncl_results.keys():
    if key in python_results.keys() and key in python_results.keys():
        try:
            assert math.isclose(
                ncl_results[key], python_results[key], rel_tol=1e-5
            )  # within 4 decimal points
            print(
                f"VALID:\n\t{key}: \n\tncl:\t\t{ncl_results[key]}\n\tpython:\t\t{python_results[key]}\n"
            )
        except Exception:
            print(
                f"INVALID:\n\t{key}: \n\t\tncl:\t\t{ncl_results[key]}\n\t\tpython:\t\t{python_results[key]}\n"
            )
            if key != "Single Point -> Invalid NCL":
                assert False

VALID:
	Boulder, Boston, Houston: 
	ncl:		1940856
	python:		1940855.984630871

VALID:
	Four Corners of Colorado: 
	ncl:		250007.6
	python:		250007.81588628562

VALID:
	Caltech, Alberta, Greenwich, Paris, Madrid: 
	ncl:		11634800
	python:		11634798.025760762

VALID:
	Crossing the Equator: 
	ncl:		114894.8
	python:		114894.73868013734

VALID:
	Crossing the Prime Meridian: 
	ncl:		54450.39
	python:		54450.41832867822

VALID:
	Half of the World: 
	ncl:		255032000
	python:		255031995.77390912

INVALID:
	Single Point -> Invalid NCL: 
		ncl:		-127516000
		python:		0.0

VALID:
	Single Degree: 
	ncl:		9401.705
	python:		9401.704877506347



### css2c

In [9]:
for key in ncl_css2c.keys():
    if key in astropy_css2c.keys():
        assert abs(ncl_css2c[key][0] - astropy_css2c[key][0]) < 0.000005
        assert abs(ncl_css2c[key][1] - astropy_css2c[key][1]) < 0.000005
        assert abs(ncl_css2c[key][2] - astropy_css2c[key][2]) < 0.000005

### csc2s

<div class="admonition alert alert-info">
    <p class="admonition-title" style="font-weight:bold">Important Note</p>
    To generate the Cartesian coordinates to test against, the NCL script for this receipt converts a range of latitude/longitude to Cartesian coordinates (with the `css2c` function). The Carestian coordinates are then converted back into latitude/longitude with the `csc2s` function. This allows the receipt to test `csc2s` across a full range of coordinates. However, NCL coordinates representing the poles (+90/-90) and the antimeridian (+180/-180) produced through this process return as an equivalent, but different value. 
    For example, an input at the pole (-90, -179) produces an output of (-90, 1) and an input of (-90,13) produces an output (-90,-167).

```
ncl 0> cart = css2c(-90, 87)
ncl 1> print(csc2s(cart(0,0), cart(1,0), cart(2,0)))
(0,0)	-90
(1,0)	-92.99999
```
The same applies for the antimerdian where, for example, an input of (-89,-180) produces an output of (-89,180)
```
ncl 4> cart = css2c(89,180)                          
ncl 5> print(csc2s(cart(0,0), cart(1,0), cart(2,0)))
(0,0)	89.00005
(1,0)	-180
```
</div>

In [10]:
# Verify Latitude/Longitude Inputs match the Latitude/Longtiude Outputs
for key in ncl_csc2s_input_output.keys():
    try:
        assert ncl_csc2s_input_output[key][0] == key[0]
        assert ncl_csc2s_input_output[key][1] == key[1]
    except Exception:
        if (
            abs(ncl_csc2s_input_output[key][0]) != 90
            and abs(ncl_csc2s_input_output[key][1]) != 180
        ):
            print(Exception)
            # Expected places where input lat/lon will not match output lat/lon in NCL
            # NCL produces flipped longitude value for +/-90 latitude, example: (90,-179)->(90,1)
            # NCL produces flipped longitude value for all latitude values when longitude is 180, example: (79,-180)->(79,180)
            assert False

In [11]:
# Verify conversions from cartesian coordinates to latitude/longtiude
for i, key in enumerate(ncl_csc2s.keys()):
    if i % 3 == 0:  # test every third point to prevent CellTimeoutError
        try:
            assert abs(key[0] - list(astropy_csc2s.keys())[i][0]) < 0.0005
            assert abs(key[1] - list(astropy_csc2s.keys())[i][1]) < 0.0005
        except Exception:
            if not math.isclose(abs(key[0]), 90) and not math.isclose(abs(key[1]), 180):
                print(abs(key[0] - list(astropy_csc2s.keys())[i][0]))
                print(abs(key[1] - list(astropy_csc2s.keys())[i][1]))
                # Expected places where input lat/lon will not match output lat/lon in NCL
                # NCL produces flipped longitude value for +/-90 latitude, example: (90,-179)->(90,1)
                # NCL produces flipped longitude value for all latitude values when longitude is 180, example: (79,-180)->(79,180)
                assert False

### gc_onarc

In [12]:
for latlon in ncl_gc_onarc.keys():
    assert ncl_gc_onarc[latlon] is python_onarc[latlon]